## 6.3 IRT(Item Response Theory)によるテスト評価

In [ ]:
# ライブラリの更新・追加導入
!pip install 'pymc>=5.9' | tail -n 1
!pip install numpyro | tail -n 1

<a href="https://colab.research.google.com/github/makaishi2/python_bayes_intro/blob/main/notebooks/6_3_IRT%E3%81%AB%E3%82%88%E3%82%8B%E3%83%86%E3%82%B9%E3%83%88%E7%B5%90%E6%9E%9C%E8%A9%95%E4%BE%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考リンク

[試験の数理 その１（問題設定とデータの生成）](https://qiita.com/takuyakubo/items/43d56725952e67032b49)

### 共通処理

In [ ]:
%matplotlib inline
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np
# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt
# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib
# pandas用ライブラリ
import pandas as pd
# データフレーム表示用関数
from IPython.display import display
# seaborn
import seaborn as sns
# 表示オプション調整
# NumPy表示形式の設定
np.set_printoptions(precision=3, floatmode='fixed')
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)
# 方眼表示ON
plt.rcParams['axes.grid'] = True
# データフレームでの表示精度
pd.options.display.float_format = '{:.3f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

In [ ]:
import pymc as pm
import arviz as az

print(f"Running on PyMC v{pm.__version__}")
print(f"Running on ArViz v{az.__version__}")

### 6.3.1 IRT(Item Response Theory)とは

#### シグモイド関数の定義

In [ ]:
def f(x):
    return 1/(1 + np.exp(-x))

#### 項目特性曲線の描画

In [ ]:
params = [
    [1, 0], [2, 0], [1, 2], [2, 2]
]

x = np.arange(-4, 4.1, 0.1)

vals = np.zeros((0, 4))

for a,b in params:
    y = f(a*(x - b))
    y1 = f(a*(1-b))
    y2 = f(a*(2-b))
    vals = np.vstack([vals, [a,b,y1, y2]])
    #print(f'a={a} b={b} f(1)={y1:.02f} f(2)={y2:.02f}')
    plt.plot(x, y, label=f'a={a} b={b}')
plt.legend()
plt.title('いろいろなパターンの項目特性曲線')
plt.show()

df = pd.DataFrame(vals, columns=['a(識別力)', 'b(困難度)', 'f(1)', 'f(2)'])
df.index = ['問題1', '問題2', '問題3', '問題4']
display(df)

### 6.3.2 問題設定

「問題」「受験者」の２軸で正解:1、不正解:0で表現したテスト結果データがある。  
このデータを使って、「各問題の困難度」と「各受験者の能力」を同時に推定する。

### 6.3.3 データ読み込み
https://qiita.com/takuyakubo/items/43d56725952e67032b49
に記載のプログラムを用いて生成したダミーデータを利用

In [ ]:
url = 'https://github.com/makaishi2/samples/raw/main/data/irt-sample.csv'
df = pd.read_csv(url, index_col=0)

# 結果確認
display(df.head())
print(df.shape)

### 6.3.4 データ加工

#### データを縦持ちに変換

In [ ]:
# melt関数で横持ち形式を縦持ち形式に変換
response_df = pd.melt(
    df.reset_index(), id_vars='index',
    var_name='question', value_name='response')

# 列名indexをuserに変換
response_df = response_df.rename({'index':'user'}, axis=1)

# 要素数の変化を確認
print(f'元データ(df): {df.shape}')
print(f'変換後データ(response_df): {response_df.shape}\n')

# 結果確認
display(response_df.head())

#### カテゴリデータの数値化

In [ ]:
# user_idx: response_dfのuser列を数値化した結果
# users: user_idxのインデックス値と元の文字列の対応
user_idx, users = pd.factorize(response_df['user'])

# question_idx: response_dfのquestion列を数値化した結果
# questions: question_idxのインデックス値と元の文字列の対応
question_idx, questions = pd.factorize(response_df['question'])

# response: 一次元化された観測値の配列
response = response_df['response'].values

# 結果確認
print('--- USER ---')
print(user_idx, len(user_idx))
print(users)
print('\n--- QUESTION ---')
print(question_idx, len(question_idx))
print(questions)
print('\n--- RESPONSE ---')
print(response, len(response))

### 6.3.5 確率モデル定義

In [ ]:
# 配列の項目定義(ユーザー軸と問題軸の2軸)
coords = {'user': users, 'question': questions}

# 確率モデルインスタンスの定義
model1 = pm.Model(coords=coords)

with model1:
    # 観測値の配列(1:正答　0:誤答)
    response_data = pm.ConstantData('response_data', response)

    # 能力値（受験者ごと）
    theta = pm.Normal('theta', mu=0.0, sigma=1.0, dims='user')

    # 識別力（設問ごと）
    a = pm.HalfNormal('a', sigma=1.0, dims='question')
    # 困難度（設問ごと）
    b = pm.Normal('b', mu=0.0, sigma=1.0, dims='question')

    # logit_pの計算 (2パラメータ・ロジスティックモデル（2PLM）)
    logit_p = pm.Deterministic(
        'logit_p', a[question_idx] * (theta[user_idx] - b[question_idx]))

    # ベルヌーイ分布の定義(1:正答　0:誤答)
    obs = pm.Bernoulli('obs', logit_p=logit_p, observed=response_data)

g = pm.model_to_graphviz(model1)
display(g)

### 6.3.6 サンプリングと結果分析
次のセルを実行するには相当の時間(15分程度)がかかるので注意して下さい。

#### サンプリング
``%%time``は時間を計測するためのマジックコマンド

In [ ]:
%%time

with model1:
#    idata1 = pm.sample(random_seed=42)
# サンプリング関数呼び出し時のパラメータ追加
    idata1 = pm.sample(random_seed=42, nuts_sampler="numpyro")

#### plot_trace関数で推論結果の確認(問題別)

In [ ]:
coords_q = {'question': ['Q001', 'Q002', 'Q003']}
az.plot_trace(
    idata1, var_names=['a', 'b'], coords=coords_q, compact=False)
plt.tight_layout();

#### plot_trace関数で推論結果の確認(受験者別)

In [ ]:
coords_u = {'user': ['USER0001', 'USER0002', 'USER0003']}
az.plot_trace(
    idata1, var_names=['theta'], coords=coords_u, compact=False)
plt.tight_layout();

### 6.3.7  詳細分析

#### サンプリング結果の統計分析(問題別)

In [ ]:
summary_a1 = az.summary(idata1, var_names=['a'])
summary_b1 = az.summary(idata1, var_names=['b'])
display(summary_a1.head())
display(summary_b1.head())

#### サンプリング結果の統計分析(受験者別)
処理にやや時間がかかります。

In [ ]:
summary_theta1 = az.summary(idata1, var_names=['theta'])
display(summary_theta1.head())

### 6.3.8 偏差値と能力値の関係

#### 偏差値と能力値の算出

In [ ]:
# 受験者ごとの正答率*100を計算し「素点」とする
df_sum1 = pd.DataFrame((df.mean(axis=1)*100), columns=['素点'])

# 素点を別変数にコピー
X = df_sum1.copy()

# 素点を偏差値のスケールに補正
X_mean, X_std = X.mean(), X.std()
X = (X-X_mean)/X_std * 10 + 50
df_sum1['偏差値'] = X

# 受験者ごとの能力値の平均を抽出
x1 = summary_theta1['mean']

# 能力値を偏差値と同じスケールに補正
x1_mean, x1_std = x1.mean(), x1.std()
x1 = (x1-x1_mean)/x1_std * 10 + 50
df_sum1['能力値'] = x1.values

# 結果の確認
display(df_sum1.head(10))

#### 偏差値との能力値の関係可視化

In [ ]:
plt.scatter(df_sum1['偏差値'], df_sum1['能力値'], s=3)
plt.title('偏差値と能力値の関係')
plt.xlabel('偏差値')
plt.ylabel('能力値');

### 6.3.9 同じ偏差値の受験者間の能力値の違い分析

#### 同じ素点(偏差値)で能力値の高い受験者と低い受験者を抽出

In [ ]:
# 1. 素点62点(偏差値51.69)の受験者を抽出
df_62_1 = df_sum1.query('素点==62')
print(f'素点62点の受験者数: {len(df_62_1)}\n')

# 2. 一番能力値の低い受験者を抽出
argmin1 = df_62_1['能力値'].values.argmin()
print('素点62点で一番能力値の低い受験者\n', df_62_1.iloc[argmin1], '\n')

# 3. 一番能力値の高い受験者を抽出
argmax1 = df_62_1['能力値'].values.argmax()
print('素点62点で一番能力値の高い受験者\n', df_62_1.iloc[argmax1])

#### 二人の受験者が正解した問題の平均困難度を計算

In [ ]:
# 二人の受験者の問題ごとの正答を抽出
w1 = df.loc[['USER0463','USER0064']]
display(w1)
w2 = w1.sum(axis=1)
print(f'w1のshape: {w1.shape}\n正解数:\n{w2}\n')

# 問題別の平均困難度
b_mean1 = az.summary(
    idata1, var_names=['b'])['mean'].values.reshape(1,-1)
print(b_mean1)
print(f'b_meanのshape: {b_mean1.shape}\n')

# 二人の受験者が解いた問題の平均困難度
w3 = (w1 * b_mean1).sum(axis=1)/w2.iloc[0]
print(w3)

#### 二人の受験者のサンプル値ごとの能力値の分布

In [ ]:
# USER0463とUSER0064の能力値のサンプルデータを抽出
w1 = idata1['posterior'].data_vars[
    'theta'].loc[:,:,['USER0463','USER0064']].values

# 軸の順番を入れ替え後、受験者を第一要素とする行列に変換
w2 = w1.transpose().reshape(2, -1)

# 能力値のスケール変更
w3 = (w2 - x1_mean)/x1_std * 10 + 50

# 受験者を列名とするデータフレームに変換
df_w3 = pd.DataFrame(w3.T, columns=['USER0463', 'USER0064'])

# Seabornのboxplot関数で可視化
sns.boxplot(df_w3)
plt.title('サンプル値ごとの能力値の分布');

### コラム　変分推論法の利用
参考リンク https://www.pymc.io/projects/examples/en/latest/variational_inference/variational_api_quickstart.html

#### 確率モデル定義

In [ ]:
# 配列の項目定義(ユーザー軸と問題軸の２軸)
coords = {'user': users, 'question': questions}

# 確率モデルインスタンスの定義
model2 = pm.Model(coords=coords)

with model2:
    # 観測値の配列(1:正答　0:誤答)
    response_data = pm.ConstantData('response_data', response)

    # 能力値（受験者ごと）
    theta = pm.Normal('theta', mu=0.0, sigma=1.0, dims='user')

    # 識別力（設問ごと）
    a = pm.HalfNormal('a', sigma=1.0, dims='question')
    # 困難度（設問ごと）
    b = pm.Normal('b', mu=0.0, sigma=1.0, dims='question')

    # logit_pの計算 (2パラメータ・ロジスティックモデル（2PLM）)
    logit_p = pm.Deterministic(
        'logit_p', a[question_idx] * (theta[user_idx] - b[question_idx]))

    # ベルヌーイ分布の定義(1:正答　0:誤答)
    obs = pm.Bernoulli('obs', logit_p=logit_p, observed=response_data)

#### 変分推論法のfit関数呼び出し

In [ ]:
%%time

with model2:
    mean_field = pm.fit(method=pm.ADVI(), n=20000,
        obj_optimizer=pm.adam())

#### 変分推論法の収束確認

In [ ]:
plt.plot(mean_field.hist);

#### 変分推論法のサンプリング

In [ ]:
%%time

idata2 = mean_field.sample(2000)

#### 偏差値と能力値の算出

In [ ]:
summary_theta2 = az.summary(idata2, var_names=['theta'])

# 本編の実習で作った変数df_sumをコピーする
df_sum2 = df_sum1.copy()

# 受験者ごとの能力値の平均を抽出
x2 = summary_theta2['mean'].values

# 能力値を偏差値と同じスケールに補正
x2_mean, x2_std = x2.mean(), x2.std()
x2 = (x2-x2_mean)/x2_std * 10 + 50
df_sum2['能力値2'] = x2

# 結果の確認
display(df_sum2.head(10))

#### 二人の受験者のサンプル値ごとの能力値の分布

In [ ]:
# USER0463とUSER0064の能力値のサンプルデータを抽出
w1 = idata2['posterior'].data_vars[
    'theta'].loc[:,:,['USER0463','USER0064']].values

# 軸の順番を入れ替え後、受験者を第一要素とする行列に変換
w2 = w1.transpose().reshape(2, -1)

# 能力値のスケール変更
w3 = (w2 - x2_mean)/x2_std * 10 + 50

# 受験者を列名とするデータフレームに変換
df_w3 = pd.DataFrame(w3.T, columns=['USER0463', 'USER0064'])

# Seabornのboxplot関数で可視化
sns.boxplot(df_w3)
plt.title('サンプル値ごとの能力値の分布');

#### バージョンの確認

In [ ]:
!pip install watermark | tail -n 1
%load_ext watermark
%watermark --iversions